## Description:
Extracts a csv file of the entire field from the imported nek output file.

In [1]:

from __future__ import annotations
from pathlib import Path
import os
_cwd = Path.cwd()
_project_root = _cwd.parent
os.chdir("..")
import yaml

from flamekit.datasets import extract_full_field_csv
from flamekit.io_fronts import Case
from mpi4py import MPI


# -----------------------------
# Find project root robustly
# -----------------------------

def find_project_root(start: Path | None = None) -> Path:
    p = (start or Path.cwd()).resolve()
    for cand in [p, *p.parents]:
        if (cand / "flamekit").is_dir():
            return cand
    raise RuntimeError("Could not find project root (folder containing 'flamekit').")


PROJECT_ROOT = find_project_root()

# INPUT PARAMETERS (from YAML)
# ============================================================
CONFIG_PATH = PROJECT_ROOT / "notebooks" / "configs" / "Extract_field.yaml"
with open(CONFIG_PATH, "r", encoding="utf-8") as f:
    CFG = yaml.safe_load(f)

def _resolve_path(value):
    p = Path(value)
    return p if p.is_absolute() else (PROJECT_ROOT / p)

TIME_STEP = int(CFG["TIME_STEP"])  # For a single time step

MULTI_TIME_STEP = bool(CFG["MULTI_TIME_STEP"])
TIME_STEP_START = int(CFG["TIME_STEP_START"])
TIME_STEP_END = int(CFG["TIME_STEP_END"])

PHI = float(CFG["PHI"])
LAT_SIZE = str(CFG["LAT_SIZE"])
T_REF = float(CFG["T_REF"])
P_REF = float(CFG["P_REF"])

POST = bool(CFG["POST"])

DATA_BASE_DIR = _resolve_path(CFG["DATA_BASE_DIR"])
OUTPUT_BASE_DIR = _resolve_path(CFG["OUTPUT_BASE_DIR"])  # keep using your unified folder convention

FILE_NAME = str(CFG["FILE_NAME"])

SCALARS = list(CFG["SCALARS"])

CANTERA_YAML = _resolve_path(CFG["CANTERA_YAML"])
CANTERA_INPUTS = [str(CANTERA_YAML), None, T_REF, P_REF]

# RUN

# ============================================================

comm = MPI.COMM_WORLD
rank = comm.rank


def _process_one_timestep(ts: int) -> None:
    case = Case(
        base_dir=OUTPUT_BASE_DIR,
        phi=PHI,
        lat_size=LAT_SIZE,
        time_step=ts,
        post=POST,
    )

    out_path = extract_full_field_csv(
        case,
        data_base_dir=DATA_BASE_DIR,
        output_base_dir=OUTPUT_BASE_DIR,
        file_name=FILE_NAME,
        scalar_names=SCALARS,
        comm=comm,
        compute_vel_jacobian=False,
        compute_vel_hessian=False,
        compute_reaction_rates=True,
        compute_progress_var=True,
        cantera_inputs=CANTERA_INPUTS,
    )

    if rank == 0:
        print(f"[rank0] Saved full field: {out_path}")


if MULTI_TIME_STEP:
    for ts in range(TIME_STEP_START, TIME_STEP_END + 1):
        if rank == 0:
            print(f"=== Processing TIME_STEP={ts} ===")
        _process_one_timestep(ts)
else:
    if rank == 0:
        print(f"=== Processing TIME_STEP={TIME_STEP} ===")
    _process_one_timestep(TIME_STEP)


=== Processing TIME_STEP=270 ===
2026-01-06 13:05:08,326 - Mesh - INFO - Initializing empty Mesh object.
2026-01-06 13:05:08,326 - Field - INFO - Initializing empty Field object
2026-01-06 13:05:08,327 - pynekread - INFO - Reading file: /media/alexandros/OS/Users/alexp/Documents/Bachelor Thesis/Code/data/phi0.40/h400x025_ref/po_postPremix0.f00001
2026-01-06 13:05:08,396 - Mesh - INFO - Initializing Mesh object from x,y,z ndarrays.
2026-01-06 13:05:08,397 - Mesh - INFO - Initializing common attributes.
2026-01-06 13:05:08,398 - Mesh - INFO - Getting vertices
2026-01-06 13:05:08,401 - Mesh - INFO - Getting edge centers
2026-01-06 13:05:08,428 - Mesh - INFO - Facet centers not available for 2D
2026-01-06 13:05:08,428 - Mesh - INFO - Mesh object initialized.
2026-01-06 13:05:08,429 - Mesh - INFO - Mesh data is of type: float64
2026-01-06 13:05:08,429 - Mesh - INFO - Elapsed time: 0.032476114s
2026-01-06 13:05:08,429 - pynekread - INFO - Reading field data
2026-01-06 13:05:08,429 - pynekrea